In [ ]:
# Data download and processing is described in paper and in repository EPC
# Provide data and code to reproduce figure 7 (then people should easily be able to reproduce all other figures as well)

In [1]:
import pickle
from helpers import *

In [2]:
data_dir = "../precip_data"

In [3]:
data_save = data_dir + '/results/'
data_save2 = data_dir + '/results/full_models/'
lsm = np.loadtxt(data_dir  + "/lsm.txt")

In [4]:
save_full = True

# DIM and Logit

In [5]:
from isodisreg import idr
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression

In [9]:
scale = 'min_max'
feature_set = 'v2+time' # for baseline model use 'v1+time'
add_time = True
season = 'JAS'
model_type = 'logit' # 'dim' or 'logit'

In [10]:
feature_set_name = feature_set[0:2]
baseline = False
if feature_set_name == 'v1':
    baseline = True

if model_type == 'logit':
    model = LogisticRegression(max_iter = 500)
else:
    model = LinearRegression()  

if baseline:
    model_name = model_type + '_base'
else:
    model_name = model_type
    
folds = np.arange(9)

crps_fold = np.zeros(len(folds))
bs_fold = np.zeros(len(folds))

In [11]:
if model_type == 'logit':

    for fold in folds:
        print(fold)

        
        bs_cv = np.zeros((19, 61))

        paths = PrecipitationDataPaths()
        feature_set_train, feature_set_test = select_data_subset(paths=paths, version=feature_set, fold=fold)
        Xtrain_prev, ytrain = load_and_concat(data_dir, fold, feature_set_train, add_time = add_time, mode = "train")
        if fold == 8:
            Xval_prev, yval = load_and_concat(data_dir, fold, feature_set_test, add_time = add_time, mode = "test")
        else:
            Xval_prev, yval = load_and_concat(data_dir, fold, feature_set_train, add_time = add_time, mode = "val")
    
        scaler_inputs = PerFeatureMeanStdScaler()

        Xtrain = scaler_inputs.fit_transform(Xtrain_prev)
        Xval = scaler_inputs.transform(Xval_prev)

        year_dim = yval.shape[0]
        ix0, ix1 = sel_season_indx(season, year_dim)

        if save_full:
            model_bs = xr.DataArray(
               np.random.rand(19, 61),
               coords=[np.arange(19) ,np.arange(-25, 35.5) ],
               dims=["lat", "lon"],
               name='var'
            )

        for i in range(19):
            for j in range(61):
                if lsm[i, j] != 0:
                    obs_train_bin = ytrain[:, i, j] > 0.2
                    obs_test_bin = yval[ix0:ix1, i, j] > 0.2
                    if np.sum(obs_train_bin) == 0:
                        probs_rain = np.zeros(len(obs_test_bin))
                        bs_cv[i, j] = np.mean((probs_rain - obs_test_bin) ** 2)
                    else:
                        clf = model.fit(Xtrain[:, :, i, j], obs_train_bin)
                        probs_rain = clf.predict_proba(Xval[ix0:ix1 , :, i, j])[:, 1]
                        bs_cv[i, j] = np.mean((probs_rain - obs_test_bin) ** 2)
                else:
                    bs_cv[i, j] = np.nan
                

        bs_fold[fold] = np.nanmean(bs_cv)
        
        if save_full:
            model_bs[:, :] = bs_cv
            model_bs.to_netcdf(data_save2 + model_name + '_bs_' + season + '_' + str(fold) +'.nc')

    np.savetxt(data_save + model_name + '_bs_'+ str(feature_set)+ '_' + season + '.txt', bs_fold)

0
1
2
3
4
5
6
7
8


In [8]:
# DIM

In [13]:
model_type

'dim'

In [15]:
if model_type == 'dim':
    for fold in folds:
        print(fold)
        crps_cv = np.zeros((19, 61))
        bs_cv = np.zeros((19, 61))
        paths = PrecipitationDataPaths()
        feature_set_train, feature_set_test = select_data_subset(paths=paths, version=feature_set, fold=fold)
        Xtrain_prev, ytrain = load_and_concat(data_dir, fold, feature_set_train, add_time = add_time, mode = "train")
        if fold == 8:
            Xval_prev, yval = load_and_concat(data_dir, fold, feature_set_test, add_time = add_time, mode = "test")
        else:
            Xval_prev, yval = load_and_concat(data_dir, fold, feature_set_train, add_time = add_time, mode = "val")
    
        scaler_inputs = PerFeatureMeanStdScaler()

        Xtrain = scaler_inputs.fit_transform(Xtrain_prev)
        Xval = scaler_inputs.transform(Xval_prev)

        year_dim = yval.shape[0]
        ix0, ix1 = sel_season_indx(season, year_dim)
        if save_full:
            model_bs = xr.DataArray(
               np.random.rand(19, 61),
               coords=[np.arange(19) ,np.arange(-25, 35.5) ],
               dims=["lat", "lon"],
               name='var'
            )
            model_crps = xr.DataArray(
               np.random.rand(19, 61),
               coords=[np.arange(19) ,np.arange(-25, 35.5) ],
               dims=["lat", "lon"],
               name='var'
            )
            
        for i in range(19):
            for j in range(61):
                if lsm[i, j] != 0:
                    ytrain_log = np.log(ytrain[:, i, j] + 0.001)
                    reg = model.fit(Xtrain[:, :, i, j], ytrain_log)
                    reg_train =  np.exp(reg.predict(Xtrain[:, :, i, j])) - 0.001
                    reg_valid = np.exp(reg.predict(Xval[ix0:ix1, :, i, j])) - 0.001
 
                    idr_output = idr(ytrain[:, i, j], pd.DataFrame({'fore': reg_train}, columns = ['fore']))
                
                    pred_idr = idr_output.predict(pd.DataFrame({'fore': reg_valid}, columns = ['fore']))
                    crps_cv[i,j] = np.mean(pred_idr.crps(yval[ix0:ix1, i, j]))
                    bs_cv[i, j] = np.mean(pred_idr.bscore(thresholds = 0.2, y = yval[ix0:ix1, i, j]))
                else:
                    crps_cv[i,j] = np.nan
                    bs_cv[i, j] = np.nan
            
        crps_fold[fold] = np.nanmean(crps_cv)
        bs_fold[fold] = np.nanmean(bs_cv)

        if save_full:
            model_bs[:, :] = bs_cv
            model_bs.to_netcdf(data_save2 + model_name + '_bs_'+ season + '_' + str(fold) +'.nc')
            model_crps[:, :] = crps_cv
            model_crps.to_netcdf(data_save2 + model_name + '_crps_'+  season + '_' + str(fold) +'.nc')

    np.savetxt(data_save + model_name + '_crps_'+ str(feature_set)+   '_' + season + '.txt', crps_fold)
    np.savetxt(data_save + model_name + '_bs_'+ str(feature_set)+  '_' + season + '.txt', bs_fold)

0
1
2
3
4
5
6
7
8


In [14]:
crps_fold

array([0.16225424, 0.15463518, 0.15574375, 0.15937833, 0.16720263,
       0.14780643, 0.15391859, 0.1516543 , 0.1480355 ])